In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.3/264.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 240.1/393.1 MB 181.8 MB/s eta 0:00:01

In [ ]:
from huggingface_hub import login
login(token="YOUR_TOKEN_HUGGINGFACE")

In [ ]:
if True:
    from unsloth import FastLanguageModel

    max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "YOUR_FINETUNED_MODEl", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

prompt = "<|im_start|>user\n{}\n<|im_end|>\n<|im_start|>assistant\n"

In [ ]:
import openai
from openai import OpenAI
# Set your API key here

client = OpenAI(api_key = "YOUR_API_KEY")

In [ ]:
def evaluate_solution(problem, candidate_solution):

    prompt = f"""
    You are a math expert reviewing the logical correctness of a step-by-step solution to a math problem.

    ### Math Problem:
    {problem}

    ### Candidate Solution:
    {candidate_solution}

    Your task is to examine each line of the Candidate Solution *individually*, checking whether it logically follows from the previous steps. If a line contains incorrect or flawed reasoning, revise **only that line** to make it logically consistent, without changing the overall structure or intent of the solution.

    Do not rewrite the entire solution — only fix the lines that are incorrect or illogical, and leave the correct lines unchanged.

    Present the final result as the corrected step-by-step solution, preserving the original format.

    ### Your revised version:
    """

    messages = [
        {"role": "system", "content": "You are a helpful and accurate math evaluator."},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or "gpt-4"
        messages=messages,
        temperature=0.0,  # for deterministic judgment
    )

    result = response.choices[0].message.content.strip()

    return result

In [ ]:
def transfer_prompt(user_input):
  prompt= f"""You are a math assistant that translates Vietnamese math problem descriptions into formal English.

The user will input a math problem in Vietnamese, possibly including mathematical expressions written in plaintext (e.g., "x^2 + y = 3").

Your tasks are:
1. Translate the entire problem into clear and formal English suitable for a math textbook or exam.
2. Convert all mathematical expressions into LaTeX format.
3. Do NOT solve the problem — only translate and format it.

Output the result as a Markdown-formatted string using LaTeX where appropriate.

### Example:

Input (Vietnamese):
"Giải phương trình bậc hai: x^2 - 5x + 6 = 0"

Output (English):
Solve the quadratic equation: $x^2 - 5x + 6 = 0$

Now, here is the user input to translate:

### User input:
{user_input}

### Your translate version:
"""
  messages = [
      {"role": "system", "content": "You are a math assistant that translates Vietnamese math problem descriptions into formal English."},
      {"role": "user", "content": prompt}
  ]

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",  # or "gpt-4"
      messages=messages,
      temperature=0.0,  # for deterministic judgment
  )

  result = response.choices[0].message.content.strip()

  return result


In [ ]:
origin_prompt = """Nếu ban đầu Jaylen có 5 củ cà rốt và 2 quả dưa chuột, và có số quả ớt chuông nhiều gấp đôi Kristin và 3 quả đậu xanh ít hơn một nửa so với Kristin, và Kristin có 2 quả ớt chuông và 20 quả đậu xanh thì tổng số rau củ đó là bao nhiêu? Jaylen có?"""

# origin_prompt = transfer_prompt(origin_prompt)

# print(origin_prompt)

# inputs = tokenizer(
# [
#     prompt.format(
#         origin_prompt,
#     )
# ], return_tensors = "pt").to("cuda")

If initially Jaylen has 5 carrots and 2 cucumbers, and has twice as many bell peppers as Kristin and 3 fewer green beans than half of Kristin's, and Kristin has 2 bell peppers and 20 green beans, then what is the total number of vegetables? How many does Jaylen have?


In [ ]:
def transfer_output(english_solution):
  prompt= f"""You are a professional math translator.

Your task is to translate the following math solution from English to Vietnamese, preserving the original mathematical notations and any LaTeX expressions.

Keep all math expressions, equations, and LaTeX syntax exactly as they are. Only translate the surrounding explanatory text and logical reasoning into natural, fluent Vietnamese.

### Math Solution in English:
{english_solution}

### Translated Solution in Vietnamese:
"""
  messages = [
      {"role": "system", "content": "You are a professional math translator."},
      {"role": "user", "content": prompt}
  ]

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",  # or "gpt-4"
      messages=messages,
      temperature=0.0,  # for deterministic judgment
  )

  result = response.choices[0].message.content.strip()

  return result


In [ ]:
from IPython.display import Markdown

def mathsolver(origin_prompt):
  origin_prompt = transfer_prompt(origin_prompt)
  inputs = tokenizer(
  [
      prompt.format(
          origin_prompt,
      )
  ], return_tensors = "pt").to("cuda")

  right_responses = []

  while len(right_responses) < 3:
      outputs = model.generate(**inputs, max_new_tokens = 1024, num_return_sequences=3, do_sample=True, use_cache = True)
      responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

      for i, response in enumerate(responses, 1):
          response = response.split('<|im_start|>assistant', 1)[1]

          # if evaluate_solution(origin_prompt, response):
          # right_responses.append(response)
          right_responses.append(evaluate_solution(origin_prompt, response))

  # for idx, right_response in enumerate(right_responses):
  #     # print("--------------------")
  #     # print(f"[Response {idx}]:\n")
  #     # print(right_response)
  #     right_response = transfer_output(right_response)
  #     display(Markdown(f"### 🧠 Phản hồi {idx + 1}:\n\n{right_response.strip()}"))
  # Gộp phản hồi thành 1 chuỗi Markdown
  markdown_output = ""
  for idx, right_response in enumerate(right_responses):
      right_response = transfer_output(right_response)
      markdown_output += f"### 🧠 Phản hồi {idx + 1}:\n\n{right_response.strip()}\n\n---\n"

  return markdown_output

In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.3 MB/s eta 0:00:00


In [ ]:
import gradio as gr

# Tạo interface
interface = gr.Interface(
    fn=mathsolver,                      # Hàm xử lý đầu vào
    inputs=gr.Textbox(label="Mô tả bài toán", lines = 10),    # Trường nhập liệu
    outputs=gr.Textbox(label="Trả lời"),    # Kết quả trả về
    title="MathSolver",
    description="F*cking Math."
)

# Chạy Gradio
interface.launch(share=True, debug=True)  # share=True để tạo link public

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d0008de511a96ff59.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
